# Setup

In [0]:
import duckietown_world as dw
import logging
dw.logger.setLevel(logging.CRITICAL)
import contracts; contracts.disable_all()

Better visualization of output

In [0]:
%%html
<style>
pre {line-height: 90%}
</style>

In [0]:

def draw_html(po, outdir=None, area=None):
    if outdir is None:
        outdir = 'out-draw_html/%s' % id(po)
    dw.draw_static(po, outdir, area=area)

    from IPython.display import IFrame, display

    iframe = IFrame(src=outdir + '/drawing.html', width='100%', height=600)
    display(iframe)

# Map representation

Let's load a map and see how data is represented inside:

In [0]:
m = dw.load_map('small_loop')
draw_html(m)

The map is a `DuckietownMap` which is an instance of `PlacedObject`. All spatially situated objects
are instances of this class.

In [0]:
type(m).mro() # see all superclasses of the object

## Children

The data is arranged in a hierarchy. We can visualize using the function `get_object_tree`. The hierarchy is more deep, but we clip it to 2 levels:

In [0]:
print(dw.get_object_tree(m, levels=2))

The children are available in the `children` variable:

In [0]:
m.children

In [0]:
m.children['tilemap'].children

You can use the notation below to get a child in a compact way:

In [0]:
tile = m['tilemap/tile-0-0']

In this case we can see how a Tile has a child `curve_left` with a child `curve` with two children `lane1` and `lane2`:

In [0]:
print(dw.get_object_tree(tile, attributes=False, levels=10))

In [0]:
draw_html(tile)

In [0]:
draw_html(tile['curve_left/curve'])

In [0]:
lane = tile['curve_left/curve/lane2']
draw_html(lane)

In [0]:
lane.width

In [0]:
lane.control_points

## Parametrization of lanes

The lane is parametrized with a parameter `beta` that interpolates among the control points.

Here we create an animation of the center point for different `beta`s.

In [0]:
import numpy as np
npoints = len(lane.control_points)
betas = list(np.linspace(-1, npoints + 1, 20))

transforms = []
for beta in betas:
    p = lane.center_point(beta)
    transform = dw.SE2Transform.from_SE2(p)
    transforms.append(transform)

ground_truth = dw.SampledSequence(betas, transforms)
lane.set_object('traveling-point', dw.PlacedObject(), ground_truth=ground_truth)
draw_html(lane)

Scrub the timeline to see the animation.